In [ ]:
import setup

setup.init()

In [ ]:
from reddit.models import RedditCommunity

import ai

In [ ]:
community_data = ai.perform_get_reddit_communities("I really like camping")

for community in community_data:
    url = community.pop("url")

    print(community)

    RedditCommunity.objects.update_or_create(url=url, defaults=community)